In [1]:
import imaplib
import email


# create an IMAP client instance
mail = imaplib.IMAP4_SSL('imap.gmail.com')


# specify the keyword to search for
email_address = input('what is your email address?: ')

password = input('what is your App password? (This is a special password under security under settings): ')
keyword_to_search = input('what keyword do you want to search?:')

# log in to the Gmail account
mail.login(email_address, password)

# select the inbox folder
mail.select('inbox')

# get the latest 150 email messages from the inbox
typ, data = mail.search(None, 'ALL')
latest_email_ids = data[0].split()[-150:]


print(f'Searching for keyword "{keyword_to_search}" in the body of the latest 150 email messages...')

count_keyword = 0

# iterate over the latest 150 email messages
for email_id in latest_email_ids:
    typ, msg_data = mail.fetch(email_id, '(RFC822)')
    for response_part in msg_data:
        if isinstance(response_part, tuple):
            # parse the email message
            msg = email.message_from_bytes(response_part[1])
            # extract the body of the email as text
            if msg.is_multipart():
                for part in msg.walk():
                    if part.get_content_type() == 'text/plain':
                        try:
                            body = part.get_payload(decode=True).decode('utf-8')
                        except UnicodeDecodeError:
                            body = part.get_payload(decode=True).decode('iso-8859-1')
                        count_keyword += body.count(keyword_to_search)
                    elif part.get_content_maintype() == 'text':
                        try:
                            body = part.get_payload(decode=True).decode('utf-8')
                        except UnicodeDecodeError:
                            body = part.get_payload(decode=True).decode('iso-8859-1')
                        count_keyword += body.count(keyword_to_search)
            else:
                try:
                    body = msg.get_payload(decode=True).decode('utf-8')
                except UnicodeDecodeError:
                    body = msg.get_payload(decode=True).decode('iso-8859-1')
                count_keyword += body.count(keyword_to_search)

# print the total number of occurrences of the keyword
print(f'The keyword "{keyword_to_search}" appears {count_keyword} time(s) in the body of the latest 150 email messages.')

what is your email address?: 5291ccc@gmail.com
what is your App password? (This is a special password under security under settings): yhtkxorkkrsydbfv
what keyword do you want to search?:alert
Searching for keyword "alert" in the body of the latest 150 email messages...
The keyword "alert" appears 212 time(s) in the body of the latest 150 email messages.
